<h1 align="center"> Principals of Biomedical Engineering </h1>
<h3 align="center"> Dr. M. Jahed </h3>
<br>
<h2 align="center"> EMG Classificatioh </h2>
<br>
<h4 align="center">Mohammad Hossein Shafieizadegan</h4>
<h4 align="center">Navid Bagheri Shouraki</h4>

# Add Required packages

In [ ]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt, iirnotch
from sklearn.preprocessing import StandardScaler

# Load Data

Load the dataset, whether our own acquired data or online free available dataset

# Signal Preprocessing

The following steps must be done:
- Bandpass Filtering
- Notch filter
- Z-score

### Bandpass filtering

We will utilize a bandpass filter regarding the frequency range of EMG signals

In [ ]:
def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

In [ ]:
fs = 1000
f_low = 20
f_high = 200

emg_signal_bandpass = butter_bandpass_filter(emg_signal, f_low, f_high, fs)

### Notch filter
Applying `50 Hz` notch filter

In [ ]:
def notch_filter(data, notch_freq, fs, Q=30):
    nyq = 0.5 * fs
    f0 = notch_freq / nyq
    b, a = iirnotch(f0, Q)
    y = filtfilt(b, a, data)
    return y

In [ ]:
notch_freq = 50
emg_signal_notch = notch_filter(emg_signal_bandpass, notch_freq, fs)

### Z-score normalization

In [ ]:
# Step 3: Z-score normalization
scaler = StandardScaler()
emg_signal_normalized = scaler.fit_transform(emg_signal_notch.reshape(-1, 1)).flatten()

# Update the DataFrame with the preprocessed signal
df['EMG_signal_preprocessed'] = emg_signal_normalized

# Signal Windowing

In [ ]:
def window_signal(emg_signal, window_length, window_step):
    
    windows = []
    num_samples = len(emg_signal)

    for start in range(0, num_samples - window_length + 1, window_step):
        end = start + window_length

        # Extract window of the signal
        windowed_signal = emg_signal[start:end]

        # Append the window to the list
        windows.append(windowed_signal)

    return windows

signal_windows = window_signal(emg_signal_normalized, window_length, window_step)

# Feature Extraction

Recommended features:
- Mean absolute value (MAV)
- Standard deviation (STD)
- Variance (VAR)
- Waveform length (WL)
- Zero crossing (ZC)
- Root mean square (RMS)
- Number of peaks (NP)
- Slope sign change (SSC)
- Correlation coefficient (Cor)
- Integrated absolute value (IAV)

#### Defining each feature

In [ ]:
import numpy as np
from scipy.stats import skew, kurtosis

def mean_absolute_value(signal):
    return np.mean(np.abs(signal))

def standard_deviation(signal):
    return np.std(signal)

def variance(signal):
    return np.var(signal)

def waveform_length(signal):
    return np.sum(np.abs(np.diff(signal)))

def zero_crossing(signal):
    return np.sum(np.diff(np.sign(signal)) != 0)

def root_mean_square(signal):
    return np.sqrt(np.mean(np.square(signal)))

def number_of_peaks(signal):
    peaks, _ = find_peaks(signal)
    return len(peaks)

def slope_sign_change(signal):
    return np.sum(np.diff(np.sign(np.diff(signal))) != 0)

def correlation_coefficient(signal1, signal2):
    return np.corrcoef(signal1, signal2)[0, 1]

def integrated_absolute_value(signal):
    return np.sum(np.abs(signal))

def find_peaks(signal, height=0):
    peaks, _ = find_peaks(signal, height=height)
    return peaks

### Applying these features to the dataset

In [ ]:
def calculate_features_for_windows(signal_windows):
    features = []

    for window in signal_windows:
        mav = mean_absolute_value(window)
        std = standard_deviation(window)
        var = variance(window)
        wl = waveform_length(window)
        zc = zero_crossing(window)
        rms = root_mean_square(window)
        npk = number_of_peaks(window)
        ssc = slope_sign_change(window)
        cor = correlation_coefficient(window, window)  # Example using the same window for correlation
        iav = integrated_absolute_value(window)

        features.append([mav, std, var, wl, zc, rms, npk, ssc, cor, iav])

    # Create a DataFrame from the features list
    columns = ['MAV', 'STD', 'VAR', 'WL', 'ZC', 'RMS', 'NP', 'SSC', 'Cor', 'IAV']
    features_df = pd.DataFrame(features, columns=columns)

    return features_df

# Classification and model training